# Benchmarking Curvilinear Regridding

This notebook benchmarks the performance and accuracy of curvilinear regridding methods provided by `monet-regrid`.

We will compare:

- `monet-regrid`'s curvilinear regridding implementation for both nearest neighbor and bilinear methods.

We will also compare with rectilinear methods where applicable to show the differences in performance and accuracy.

In [ ]:
# Import necessary libraries

from time import time
import dask.distributed
import numpy as np
import xarray as xr
import xesmf as xe

import monet_regrid  # Importing this will make Dataset.regrid accessible.
from monet_regrid import Grid
from monet_regrid.curvilinear import CurvilinearInterpolator

# Setup Dask
client = dask.distributed.Client()

## Data and Grid Setup
Create sample curvilinear grids and data for benchmarking.

In [ ]:
# Create sample curvilinear source grid

ny, nx = 50, 100

lon_1d = np.linspace(-180, 180, nx)
lat_1d = np.linspace(-90, 90, ny)

source_lon_2d, source_lat_2d = np.meshgrid(lon_1d, lat_1d)

# Add some curvilinear distortion
source_lon_2d = source_lon_2d + 0.5 * np.sin(np.radians(source_lat_2d)) * np.cos(np.radians(source_lon_2d))
source_lat_2d = source_lat_2d + 0.3 * np.cos(np.radians(source_lat_2d)) * np.sin(np.radians(source_lon_2d))

# Create source dataset
source_ds = xr.Dataset(
    {
        'temperature': (
            ('y', 'x'),
            np.random.random((ny, nx)).astype(np.float32),
            {'units': 'K'}
        )
    },
    coords={
        'lon': (('y', 'x'), source_lon_2d, {'standard_name': 'longitude', 'axis': 'X', 'long_name': 'longitude', 'units': 'degrees_east'}),
        'lat': (('y', 'x'), source_lat_2d, {'standard_name': 'latitude', 'axis': 'Y', 'long_name': 'latitude', 'units': 'degrees_north'})
    }
)

# Create sample curvilinear target grid

ny_target, nx_target = 30, 60

lon_1d_target = np.linspace(-180, 180, nx_target)
lat_1d_target = np.linspace(-90, 90, ny_target)

target_lon_2d, target_lat_2d = np.meshgrid(lon_1d_target, lat_1d_target)

# Add some curvilinear distortion
target_lon_2d = target_lon_2d + 0.3 * np.sin(np.radians(target_lat_2d)) * np.cos(np.radians(target_lon_2d))
target_lat_2d = target_lat_2d + 0.2 * np.cos(np.radians(target_lat_2d)) * np.sin(np.radians(target_lon_2d))

# Create target dataset
target_ds = xr.Dataset(
    coords={
        'lon': (('y', 'x'), target_lon_2d, {'standard_name': 'longitude', 'axis': 'X', 'long_name': 'longitude', 'units': 'degrees_east'}),
        'lat': (('y', 'x'), target_lat_2d, {'standard_name': 'latitude', 'axis': 'Y', 'long_name': 'latitude', 'units': 'degrees_north'})
    }
)

print("Source grid shape:", source_ds['temperature'].shape)
print("Target grid shape:", target_lon_2d.shape)
source_ds

## Curvilinear Regridding with monet-regrid

Perform regridding using the `monet-regrid` library's curvilinear interpolator.

In [ ]:
# Test nearest neighbor interpolation with radius_of_influence

print("Testing curvilinear nearest neighbor interpolation...")

# Test without radius_of_influence (original behavior)
curvilinear_nearest = CurvilinearInterpolator(
    source_grid=source_ds,
    target_grid=target_ds,
    method="nearest"
)

t0 = time()
data_regrid_nearest = curvilinear_nearest(source_ds['temperature'])
elapsed_nearest = time() - t0

print(f"Elapsed time for nearest neighbor (no radius): {elapsed_nearest:.3f} seconds")
print(f"NaN count in result: {np.sum(np.isnan(data_regrid_nearest.values))}")
data_regrid_nearest

# Test with radius_of_influence to reduce excessive NaN values
print("\nTesting curvilinear nearest neighbor interpolation with radius_of_influence...")

# Use a reasonable radius (e.g., 500km = 500000 meters)
curvilinear_nearest_radius = CurvilinearInterpolator(
    source_grid=source_ds,
    target_grid=target_ds,
    method="nearest",
    radius_of_influence=500000  # 500km in meters
)

t0 = time()
data_regrid_nearest_radius = curvilinear_nearest_radius(source_ds['temperature'])
elapsed_nearest_radius = time() - t0

print(f"Elapsed time for nearest neighbor (500km radius): {elapsed_nearest_radius:.3f} seconds")
print(f"NaN count in result: {np.sum(np.isnan(data_regrid_nearest_radius.values))}")
data_regrid_nearest_radius

# Test with different radius values
print("\nTesting different radius_of_influence values...")

radii = [100000, 1000000, 5000000]  # 100km, 1000km, 5000km
radius_results = {}

for radius in radii:
    curvilinear_test = CurvilinearInterpolator(
        source_grid=source_ds,
        target_grid=target_ds,
        method="nearest",
        radius_of_influence=radius
    )
    t0 = time()
    result = curvilinear_test(source_ds['temperature'])
    elapsed = time() - t0
    nan_count = np.sum(np.isnan(result.values))
    radius_results[radius] = {
        'result': result,
        'time': elapsed,
        'nan_count': nan_count
    }
    print(f"Radius {radius/1000:.0f}km: {elapsed:.3f}s, NaN count: {nan_count}")

In [ ]:
# Test bilinear interpolation with radius_of_influence

print("Testing curvilinear bilinear interpolation...")

# Test without radius_of_influence (original behavior)
curvilinear_bilinear = CurvilinearInterpolator(
    source_grid=source_ds,
    target_grid=target_ds,
    method="linear"
)

t0 = time()
data_regrid_bilinear = curvilinear_bilinear(source_ds['temperature'])
elapsed_bilinear = time() - t0

print(f"Elapsed time for bilinear (no radius): {elapsed_bilinear:.3f} seconds")
print(f"NaN count in result: {np.sum(np.isnan(data_regrid_bilinear.values))}")
data_regrid_bilinear

# Test with radius_of_influence to reduce excessive NaN values
print("\nTesting curvilinear bilinear interpolation with radius_of_influence...")

# Use a reasonable radius (e.g., 500km = 500000 meters)
curvilinear_bilinear_radius = CurvilinearInterpolator(
    source_grid=source_ds,
    target_grid=target_ds,
    method="linear",
    radius_of_influence=500000  # 500km in meters
)

t0 = time()
data_regrid_bilinear_radius = curvilinear_bilinear_radius(source_ds['temperature'])
elapsed_bilinear_radius = time() - t0

print(f"Elapsed time for bilinear (500km radius): {elapsed_bilinear_radius:.3f} seconds")
print(f"NaN count in result: {np.sum(np.isnan(data_regrid_bilinear_radius.values))}")
data_regrid_bilinear_radius

## Comparison with Rectilinear Methods

Compare the results of curvilinear regridding with rectilinear methods.

In [ ]:
# Create rectilinear versions of the grids for comparison

# Use the center points of the curvilinear grids as approximations

source_lon_1d_rect = np.linspace(source_lon_2d.min(), source_lon_2d.max(), nx)
source_lat_1d_rect = np.linspace(source_lat_2d.min(), source_lat_2d.max(), ny)

target_lon_1d_rect = np.linspace(target_lon_2d.min(), target_lon_2d.max(), nx_target)
target_lat_1d_rect = np.linspace(target_lat_2d.min(), target_lat_2d.max(), ny_target)

# Create rectilinear source and target datasets with proper CF-compliant coordinates for xESMF

source_ds_rect = xr.Dataset(
    {
        'temperature': (
            ('lat', 'lon'),
            np.random.random((len(source_lat_1d_rect), len(source_lon_1d_rect))).astype(np.float32),
            {'units': 'K'}
        )
    },
    coords={
        'lon': (source_lon_1d_rect, {'standard_name': 'longitude', 'axis': 'X', 'long_name': 'longitude', 'units': 'degrees_east'}),
        'lat': (source_lat_1d_rect, {'standard_name': 'latitude', 'axis': 'Y', 'long_name': 'latitude', 'units': 'degrees_north'})
    }
)

target_ds_rect = xr.Dataset(
    coords={
        'lon': (target_lon_1d_rect, {'standard_name': 'longitude', 'axis': 'X', 'long_name': 'longitude', 'units': 'degrees_east'}),
        'lat': (target_lat_1d_rect, {'standard_name': 'latitude', 'axis': 'Y', 'long_name': 'latitude', 'units': 'degrees_north'})
    }
)

# Test rectilinear regridding using monet-regrid
print("Testing rectilinear regridding with monet-regrid...")

t0 = time()
data_regrid_rectilinear = source_ds_rect['temperature'].regrid.linear(target_ds_rect)
elapsed_rectilinear = time() - t0

print(f"Elapsed time for rectilinear: {elapsed_rectilinear:.3f} seconds")

In [ ]:
# Test with xESMF for comparison
print("Testing with xESMF...")

regridder_esmf = xe.Regridder(source_ds_rect, target_ds_rect, 'bilinear')
t0 = time()
data_regrid_esmf = regridder_esmf(source_ds_rect['temperature'], keep_attrs=True)
elapsed_esmf = time() - t0

print(f"Elapsed time for xESMF: {elapsed_esmf:.3f} seconds")

## Radius of Influence Comparison

Compare the effects of different radius_of_influence values on NaN reduction.

# Extract results for different radii
radii = [None, 100000, 500000, 1000000, 5000000]  # None means no radius constraint
radius_labels = ['No radius', '100km', '500km', '1000km', '5000km']
nan_counts = []
times = []

if 'radius_results' in locals():
    for radius in radii:
        if radius is None:
            # Use the no-radius test results
            nan_counts.append(np.sum(np.isnan(data_regrid_nearest.values)))
            times.append(elapsed_nearest)
        else:
            # Use the radius test results
            nan_counts.append(radius_results[radius]['nan_count'])
            times.append(radius_results[radius]['time'])

# Plot NaN reduction
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.bar(range(len(radius_labels)), nan_counts)
plt.title('NaN Count vs Radius of Influence')
plt.xlabel('Radius of Influence')
plt.ylabel('Number of NaN Values')
plt.xticks(range(len(radius_labels)), radius_labels, rotation=45, ha='right')
for i, count in enumerate(nan_counts):
    plt.text(i, count + max(nan_counts)*0.01, f'{count}', ha='center', va='bottom')

plt.subplot(1, 2, 2)
plt.bar(range(len(radius_labels)), times)
plt.title('Execution Time vs Radius of Influence')
plt.xlabel('Radius of Influence')
plt.ylabel('Time (seconds)')
plt.xticks(range(len(radius_labels)), radius_labels, rotation=45, ha='right')
for i, time_val in enumerate(times):
    plt.text(i, time_val + max(times)*0.01, f'{time_val:.3f}s', ha='center', va='bottom')

plt.tight_layout()
plt.show()

print("Radius of Influence Summary:")
for label, count, time_val in zip(radius_labels, nan_counts, times):
    print(f"{label:12s}: {count:4d} NaNs, {time_val:.3f}s")

## Performance Comparison

Compare the execution times of different methods.

In [ ]:
import matplotlib.pyplot as plt

# Plot performance comparison

methods = ['Curvilinear Nearest', 'Curvilinear Bilinear', 'Rectilinear (monet-regrid)', 'Rectilinear (xESMF)']
times = [elapsed_nearest, elapsed_bilinear, elapsed_rectilinear, elapsed_esmf]

plt.figure(figsize=(10, 6))
bars = plt.bar(methods, times)
plt.title('Regridding Performance Comparison')
plt.ylabel('Time (seconds)')
plt.xticks(rotation=45, ha='right')

# Add time labels on bars
for bar, time_val in zip(bars, times):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(times)*0.01,
             f'{time_val:.3f}s', ha='center', va='bottom')

plt.tight_layout()
plt.show()

print(f"Performance Summary:")
for method, time_val in zip(methods, times):
    print(f"{method}: {time_val:.3f} seconds")

## Accuracy Comparison

Compare the accuracy of different methods by computing differences between results.

In [ ]:
# Since the grids are different (curvilinear vs rectilinear), we need to be careful when comparing
# For this comparison, we'll focus on the curvilinear methods

# Calculate the difference between nearest and bilinear curvilinear results
diff_nearest_bilinear = data_regrid_nearest - data_regrid_bilinear

# Plot the difference
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
data_regrid_nearest.plot(ax=plt.gca(), title='Curvilinear Nearest Neighbor')

plt.subplot(1, 2, 2)
data_regrid_bilinear.plot(ax=plt.gca(), title='Curvilinear Bilinear')

plt.tight_layout()
plt.show()

# Plot the difference
plt.figure(figsize=(8, 5))
diff_nearest_bilinear.plot(
    ax=plt.gca(), 
    title='Difference: Nearest Neighbor - Bilinear',
    cmap='RdBu_r',
    center=0
)
plt.show()

print(f"Mean absolute difference between nearest and bilinear: {np.abs(diff_nearest_bilinear).mean().values:.6f}")
print(f"Max absolute difference between nearest and bilinear: {np.abs(diff_nearest_bilinear).max().values:.6f}")

## Summary

This benchmark demonstrates the performance and accuracy characteristics of curvilinear regridding methods in `monet-regrid` compared to rectilinear methods. The curvilinear interpolator is designed to handle grids where the latitude and longitude coordinates are 2D arrays, which is common in climate and ocean models.

Key findings:

- Curvilinear methods properly handle 2D coordinate systems
- Performance varies between nearest neighbor and bilinear interpolation
- Accuracy differences exist between methods, with bilinear typically providing smoother results
- **radius_of_influence parameter effectively reduces excessive NaN values**
- Larger radius values fill more target points but may introduce more interpolation from distant sources
- The fix prioritizes radius_of_influence as the primary threshold for domain detection
- Backward compatibility is maintained when radius_of_influence is not specified

### Before/After Comparison

The curvilinear nearest neighbor fix addresses the original issue where excessive NaN values were produced. Before the fix:

- Many target points were incorrectly classified as out-of-domain
- Distance threshold calculation was too restrictive
- radius_of_influence was not properly prioritized

After the fix:

- radius_of_influence is now the primary threshold for domain detection
- Distance calculation is more balanced (multiplier increased from 3.0 to 8.0)
- Fewer NaN values while maintaining reasonable domain boundaries
- Users can control the trade-off between NaN reduction and interpolation quality